In [220]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?i=computers&bbn=429886031&rh=n%3A429886031%2Cp_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&dc&page=2&qid=1698924010&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-47',
 '/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHHH4T4/ref=sr_1_35?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35',
 '/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHTPQBN/ref=sr_1_42?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-42',
 '/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43',
 '/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031

In [221]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Macbook'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-47
2 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHHH4T4/ref=sr_1_35?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
3 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHTPQBN/ref=sr_1_42?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-42
4 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
5 / 240
/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A41

42 / 240
/2019-Apple-MacBook-Pro-16/dp/B0BB3BK6SG/ref=sr_1_46?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-46
43 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_37?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
44 / 240
/Apple-MacBook-Air-Late-2018/dp/B07RSDV1SV/ref=sr_1_45?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-45
45 / 240
/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_38?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-38
46 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3C14BMY/ref=sr_1_26?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-26
47 / 240
/Ap

83 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_30?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-30
84 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_37?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
85 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-27
86 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_34?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
87 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&s

124 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-27
125 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_34?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
126 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
127 / 240
/Apple-MacBook-Chip-13-inch-256GB/dp/B08XY2779H/ref=sr_1_33?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-33
128 / 240
/Apple-MacBook-laptop-12%E2%80%91core-30%E2%80%91core/dp/B0BSHRZ6DS/ref=sr_1_31?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=41915203

164 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_30?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-30
165 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3C14BMY/ref=sr_1_26?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-26
166 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHVC2QW/ref=sr_1_32?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-32
167 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WTNQ6/ref=sr_1_25?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-25
168 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=comput

205 / 240
/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-48
206 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BG5DJ7/ref=sr_1_43?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
207 / 240
/2019-Apple-MacBook-Pro-16/dp/B0BB3BK6SG/ref=sr_1_46?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-46
208 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_35?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
209 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WTNQ6/ref=sr_1_26?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=c

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Ear placement,Charging time,Style,Shape,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1224,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1225,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1226,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN


In [222]:
df.Search.value_counts()

iPhone         609
AirPods        255
iPad           168
Macbook        107
Apple Watch     79
Name: Search, dtype: int64

In [223]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=iphone&i=electronics&rh=n%3A560820%2Cp_72%3A419156031&dc&page=2&qid=1698924321&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25',
 '/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_47?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47',
 '/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41',
 '/Apple-iPhone-14-Plus-128/dp/B0BDJY2DFH/ref=sr_1_46?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46',
 '/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33',
 '/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39',
 '/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_38?ke

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPhone'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
2 / 240
/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_47?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47
3 / 240
/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41
4 / 240
/Apple-iPhone-14-Plus-128/dp/B0BDJY2DFH/ref=sr_1_46?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46
5 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
6 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39
7 / 240
/Apple-iPhone-mini-128GB-Blac

52 / 240
/Apple-iPhone-mini-64GB-Black/dp/B08PCF2P4R/ref=sr_1_35?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-35
53 / 240
/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_38?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-38
54 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39
55 / 240
/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41
56 / 240
/Apple-iPhone-14-128-GB/dp/B0BDJJKPT2/ref=sr_1_34?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-34
57 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
58 / 240
/Apple-iPhone-15-Pro-256/dp/B0CHX5R7K3/ref=s

103 / 240
/Apple-iPhone-12-128GB-Black/dp/B08PCGL5TH/ref=sr_1_48?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-48
104 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
105 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
106 / 240
/Apple-iPhone-15-Pro-256/dp/B0CHX5R7K3/ref=sr_1_32?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-32
107 / 240
/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
108 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_40?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-40
109 / 240
/Apple-iPhone-128GB-Pacific-Blue/dp/B08PCB8Q

In [ ]:
df.Search.value_counts()

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=ipad&i=electronics&page=2&crid=2SLN76UU27TLB&qid=1698924622&sprefix=ipad%2Celectronics%2C98&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


In [ ]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPad'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


In [ ]:
df.Search.value_counts()

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=AirPods&i=electronics&rh=n%3A4085731%2Cp_72%3A419156031&dc&page=2&qid=1698924837&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


In [ ]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'AirPods'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


In [ ]:
df.Search.value_counts()

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=apple+watch&i=electronics&rh=n%3A560798%2Cp_72%3A419156031&dc&page=2&qid=1698925012&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


In [ ]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Apple Watch'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


In [ ]:
df.Search.value_counts()

In [ ]:
Mac
iPhone
iPad
AirPods
Watch


In [175]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

class AmazonScraper:
    def __init__(
        self,
        headless=True,
        load_images=False,
        chromedriver_path="./chromedriver",
        window_size=(700,900),
        ):
        options = webdriver.ChromeOptions()
        
        if headless:
            options.add_argument("--headless")
            options.add_argument("--disable-gpu")
        
        if not load_images:
            prefs = {"profile.managed_default_content_settings.images": 2}
            options.add_experimental_option("prefs", prefs)
        
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(
            executable_path=chromedriver_path, options=options
        )
        self.driver.set_window_size(*window_size)
            

In [180]:
scraper = AmazonScraper(headless=False)

In [188]:
scraper.driver.get('https://www.amazon.co.uk/Apple-iPhone-11-64GB-White/dp/B08L6XKFRY/ref=cm_cr_arp_d_product_top?ie=UTF8'
                  )

In [192]:
scraper.driver.find_element_by_class_name('a-size-base review-text').text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a-size-base review-text"}
  (Session info: chrome=119.0.6045.105)
